In [55]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
        builder. \
        config('spark.ui.port','0'). \
        config('spark.shuffle.useOldFetchProtocol','true'). \
        config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
        enableHiveSupport(). \
        master('yarn'). \
        getOrCreate()

In [56]:
loans_def_raw_df=spark.read\
.format("csv")\
.option("header",True)\
.option("inferSchema",True) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [57]:
loans_def_raw_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
9cb79aa7323e81be1...,2.0,0.0,0.0,0.0,0.0,0.0,11.0,null
0dd2bbc517e3c8f9e...,0.0,0.0,1.0,1.0,3.0,0.0,null,115.0
458458599d3df3bfc...,0.0,0.0,1.0,1.0,1.0,0.0,null,76.0
05ea141ec28b5c7f7...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
aac68850fdac09fd0...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
3a423e4589e89f429...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
f1efcf7dfbfef21be...,0.0,0.0,0.0,0.0,1.0,0.0,null,null
c89986155a070db2e...,1.0,0.0,0.0,0.0,1.0,15.0,5.0,null
118dc629b6e134419...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
a86fa4b7493708333...,0.0,0.0,0.0,0.0,0.0,0.0,null,null


In [58]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [59]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [60]:
spark.sql("select distinct(delinq_2yrs) from loan_defaulters")

delinq_2yrs
1.0
I bike to work on...
271 monthly payme...
183xx
VISA and AMEX cre...
etc. and I feel t...
AZ
017xx
923xx
446xx


In [61]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+------------------+-------+
|       delinq_2yrs|  total|
+------------------+-------+
|               0.0|1838878|
|               1.0| 281335|
|               2.0|  81285|
|               3.0|  29539|
|               4.0|  13179|
|               5.0|   6599|
|               6.0|   3717|
|               7.0|   2062|
|               8.0|   1223|
|               9.0|    818|
|              10.0|    556|
|              11.0|    363|
|              12.0|    264|
|              13.0|    165|
|              14.0|    120|
|              15.0|     87|
|              null|     63|
|              16.0|     55|
|              17.0|     30|
|              18.0|     30|
|              19.0|     23|
|              20.0|     17|
|              21.0|     12|
|                CA|      8|
|                TX|      6|
|                IL|      5|
|debt_consolidation|      5|
|    small_business|      5|
|              22.0|      5|
|              24.0|      4|
|                FL|      4|
|             

In [62]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float,pub_rec float,pub_rec_bankruptcies float,inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float" 

In [63]:
loans_def_raw_df=spark.read\
.format("csv")\
.option("header",True)\
.schema(loan_defaulters_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [64]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [65]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- delinq_amnt: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- pub_rec_bankruptcies: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- mths_since_last_delinq: float (nullable = true)
 |-- mths_since_last_record: float (nullable = true)



In [66]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       17.0|     30|
|       18.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       29.0|      2|
|       25.0|      2|
|       30.0|      2|
|       3.44|      2|
|       23.0|      2|
|       36.0|      1|
|       32.0|      1|
|       5.52|      1|
|      22.62|      1|
|       35.0|      1|
|       27.0|      1|
|       8.56|      1|
|      10.58|      1|
|      16.42|      1|
+-----------+-------+
only showing top 40 rows



In [67]:
from pyspark.sql.functions import col

In [68]:
loans_def_processed_df=loans_def_raw_df.withColumn("delinq_2yrs",col("delinq_2yrs").cast("integer")).fillna(0,subset=["delinq_2yrs"])

In [69]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

In [70]:
spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null")

count(1)
0


In [71]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|          0|1839141|
|          1| 281337|
|          2|  81285|
|          3|  29545|
|          4|  13180|
|          5|   6601|
|          6|   3719|
|          7|   2063|
|          8|   1226|
|          9|    821|
|         10|    558|
|         11|    363|
|         12|    266|
|         13|    167|
|         14|    123|
|         15|     90|
|         16|     56|
|         17|     33|
|         18|     32|
|         19|     24|
|         20|     19|
|         21|     16|
|         22|      7|
|         24|      6|
|         23|      5|
|         26|      4|
|         25|      2|
|         29|      2|
|         30|      2|
|         27|      1|
|         39|      1|
|         28|      1|
|         32|      1|
|         35|      1|
|         58|      1|
|         42|      1|
|         36|      1|
+-----------+-------+



In [82]:
loans_def_dealing_df = spark.sql("select member_id, delinq_2yrs,delinq_amnt, int(mths_since_last_delinq) from loan_defaulters where delinq_2yrs > 0 or mths_since_last_delinq > 0")

In [83]:
loans_def_dealing_df.count()

1106163

In [87]:
loans_def_records_eng_df = spark.sql("select member_id from loan_defaulters where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [89]:
loans_def_records_eng_df.count()

1070125

In [ ]:
loans_def_dealing_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/cleaned/loans_defaulters_deling_parquet") \
.save()

In [ ]:
loans_def_dealing_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv006222/myproject/cleaned/loans_defaulters_deling_parquet") \
.save()